#  Reconnaissance de visages

# datasets

detec visage dans image

In [4]:
import cv2 as cv
def face_detect_demo():
#gary
    gary = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# haarcascade_frontalface_alt2.xml
    face_detect = cv.CascadeClassifier('C:\\Users\\josephine\\AppData\\Local\\Programs\\Python\\Python37\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_alt2.xml')
# detcte plusieur visage
    face = face_detect.detectMultiScale(gary)
# utilise une rectangle pour detec visage
    for x,y,w,h in face:
       cv.rectangle(img,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
    cv.imshow('result',img)
img = cv.imread('D:\\deep learning\\opencv\\photo.jpg')
face_detect_demo()
while True:
     if ord('q') == cv.waitKey(0):
         break
cv.destroyAllWindows()

capture et sauvager les visage ce que obtenir dans le camera

In [3]:
import cv2 as cv
#utilise un fonction pour dectec le visage
def face_detect_demo(img,count):
    #gary = cv.cvtColor(img,cv.COLOR_RGB2GRAY)
    face_detect = cv.CascadeClassifier('C:\\Users\\josephine\\AppData\\Local\\Programs\\Python\\Python37\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_alt2.xml')
    face = face_detect.detectMultiScale(img)#gary
    # utilise une rectangle pour detec visage et sauvage le  visage dans rectangle
    for x,y,w,h in face:
       img = cv.rectangle(img,(x-20,y-20),(x+w+20,y+h+20),color=(0,0,255),thickness=2)
       image = cv.resize(img[y-20:y + h+20, x-20:x + w+20], (300, 300))#gary
       cv.imwrite("D:/User." + str(count) + '.jpg', image)
    cv.imshow('result',img)

#lire camera
cap = cv.VideoCapture(0)
cap.read()
count = 0
#utilise un boucle 
while True:
    f,frame = cap.read()
    if not f:
        break
    else:
        count =count + 1
    #utilise fonction
    face_detect_demo(frame,count)
    #si clique q,nous avons sorti le boucle while
    if ord('q') == cv.waitKey(10):
        break
cv.destroyAllWindows()
cap.release()

# apprantisage 

utilise pytroch pour apprantisage et valide

In [ ]:
import torch
import matplotlib.pyplot as plt
from torchsummary import summary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os

#create a CNN
#avec deux couche de convolution, trois couche complètement connectée, deux couche de pooling
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.name = "CNN for face"
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2,2)

        #self.conv3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5)
        #self.pool3 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(32*22*22,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))    # input(3, 100, 100) output(16, 96, 96)
        x = self.pool1(x)            # output(16, 48, 48)
        x = F.relu(self.conv2(x))    # output(32, 44, 44)
        x = self.pool2(x)            # output(32, 22, 22)
        #x = F.relu(self.conv3(x))    # output(48, 18, 18)
        #x = self.pool3(x)            # output(48, 9, 9)

        x = torch.flatten(x, 1)     #aplatissement

        x = F.relu(self.fc1(x))      # output(120)
        x = F.relu(self.fc2(x))      # output(84)
        x = self.fc3(x)          # output(10)
        return x
    
#root
#train_dir='/content/drive/MyDrive/imageLUO'
train_dir = './imagLUO/train'

#size
im_w, im_h = 100, 100
transform = transforms.Compose([
                 transforms.Resize((im_w, im_h)),
                 transforms.ToTensor(),   #car la transformation qui suit porte sur des tenseurs
                ])

#import data
full_dataset = datasets.ImageFolder(train_dir, transform=transform)
#split dataset for train/test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#batch et loader data
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

#put data into graphics card
#device = "cpu" #ou "cuda"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
summary(model, (3, im_w, im_h))
optimizer = optim.Adam(model.parameters(), lr=0.001)    #utilisé par l'apprentissage (lr <-> learning rate)
criterion = nn.CrossEntropyLoss()       #critère d'erreur en sortie (utilisé par l'apprentissage et le test)

#function traning
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0.0
    correct = 0
    data_len = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()                              #initialisation des gradients
        output = model(data)                               #activation du réseau
        loss = criterion(output, target)                   #calcul de l'erreur de sortie
        loss.backward()                                    #rétro-propagation du gradient
        optimizer.step()                                   #adaptation des poids des neurones
        train_loss += loss.item()# * data.size(0)

        pred = output.argmax(dim=1, keepdim=True)          # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{}]\tTrain loss: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, loss.item()))
    train_loss /= data_len
    accuracy = 100. * correct / data_len
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain accuracy: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, 100. * batch_idx / data_len, accuracy))
    return train_loss, accuracy

#function test
def test(model, device, test_loader):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target)#.item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average test loss: {:.4f}, Test accuracy: ({:.2f}%)\n'.format(loss, accuracy))
    return loss, accuracy

#function for design
def matplot_loss(train_losses,test_losses,train_accuracies,test_accuracies):
  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)

  plt.title("loss vs learning epochs on face dataset")
  plt.ylabel("loss")
  plt.plot(train_losses, label='Train')
  plt.plot(test_losses, label='Test')
  plt.legend(loc="upper right")
  plt.subplot(2, 1, 2)

  plt.title("accuracy vs learning epochs on face dataset")
  plt.xlabel("epochs")
  plt.ylabel("accuracy")
  plt.plot(train_accuracies, label='Train')
  plt.plot(test_accuracies, label='Test')
  plt.legend(loc="lower right")
  plt.show()

#begin traning
epochs = 50
min_accuracy = 0
test_losses, test_accuracies, train_losses, train_accuracies = [], [], [], []
print('Learning (please wait)...')
for epoch in range(1, epochs + 1):
    train_loss, train_accuracy = train(model, device, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, device, test_loader)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    #save most better poids
    if test_accuracy > min_accuracy:
      folder = './models/save_model'
      if not os.path.exists(folder):
        os.mkdir('./models/save_model')
      min_accuracy=train_accuracy
      torch.save(model.state_dict(),'./models/save_model/best_model.pth')
      print('save epoch'+str(epoch))
print('Done')
matplot_loss(train_losses,test_losses,train_accuracies,test_accuracies)

Tests de différents paramètres du RNC

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def predict(model, image):
    image = image.resize((100, 100))
    np_img = np.array(image)                        # conversion en tableau numpy
    image_tensor = torch.from_numpy(np_img)         # conversion en tenseur PyTorch
    image_tensor = image_tensor.permute(2, 0, 1)    # nouvel ordre des dimensions
    image_tensor = image_tensor.reshape([1, 3, np_img.shape[0], np_img.shape[1]]) # ajout d'une dimension (cf + haut)
    output = model(image_tensor.float())        # application de l'image en entrée du réseau
    index = output.data.cpu().numpy().argmax()
    return index
 
image_file1 = '/content/drive/MyDrive/deep learning/imageTest/EmmanuelTest.jpg'
image_file2 = '/content/drive/MyDrive/deep learning/imageTest/HuangXingjiantest.jpg'
image_file3 = '/content/drive/MyDrive/deep learning/imageTest/LUOWenqicolortest.jpg'
image_file4 = '/content/drive/MyDrive/deep learning/imageTest/ValexTest.jpg'
image_file5 = '/content/drive/MyDrive/deep learning/imageTest/unknownTest.jpg'

model = Net()                                    # re-création d'un réseau de neurones (identique au précédent)
model.load_state_dict(torch.load('./models/save_model/best_model.pth'))    # chargement du modèle
model.eval()

images = []
img1 = Image.open(image_file1)
img2 = Image.open(image_file2)
img3 = Image.open(image_file3)
img4 = Image.open(image_file4)
img5 = Image.open(image_file5)
images.append(img1)
images.append(img2)
images.append(img3)
images.append(img4)
images.append(img5)

for i in range(5):
  #programme principal
  true_index = i            #classe "Emmanuel" : incdex = 0 classe "HUANGXinjian" : index = 1 ; classe "LUOWenqi" ; index = 2 ：classe "Valex" index = 3 : classe "unknown" index = 4

  index = predict(model, images[i])
  print('index : ', index)

  #affichage du résultat sur l'image
  classe = []    #nom des classes
  classe.append('Emmanuel')
  classe.append('HUANGXingjian')
  classe.append('LUOWenqi')
  classe.append('Valex')
  classe.append('unknown')
  if(true_index == index):
      res = True
  else:
      res = False
  test_loss, test_accuracy = test(model, device, test_loader)
  plt.figure(figsize=(8, 8))
  plt.subplot(5, 1, i+1)
  plt.title('classe : ' + str(classe[index]) + ' (' + str(res) + ')')
  plt.axis('off')
  plt.imshow(images[i])

# detection visage

change FDDB dans pascal VOC fomat

In [ ]:
#https://github.com/penolove/FDDB_DataSet_4_faster_rcnn

test de l’algorithme MobileNetV2-SSD avec image et video

In [ ]:
#image
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.utils.misc import Timer
import cv2
import matplotlib.pyplot as plt

net_type = 'mb2-ssd-lite'
model_path = '/content/drive/MyDrive/deep learning/MobileNetSSD/mb2-ssd-lite-mp-0_686.pth'
label_path = '/content/drive/MyDrive/deep learning/MobileNetSSD/voc-model-labels.txt'
image_path = '/content/drive/MyDrive/FDDB_2010/JPEGImages/000000.jpg'

class_names = [name.strip() for name in open(label_path).readlines()]

if net_type == 'vgg16-ssd':
    net = create_vgg_ssd(len(class_names), is_test=True)
elif net_type == 'mb1-ssd':
    net = create_mobilenetv1_ssd(len(class_names), is_test=True)
elif net_type == 'mb1-ssd-lite':
    net = create_mobilenetv1_ssd_lite(len(class_names), is_test=True)
elif net_type == 'mb2-ssd-lite':
    net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)
elif net_type == 'mb3-large-ssd-lite':
    net = create_mobilenetv3_large_ssd_lite(len(class_names), is_test=True)
elif net_type == 'mb3-small-ssd-lite':
    net = create_mobilenetv3_small_ssd_lite(len(class_names), is_test=True)
elif net_type == 'sq-ssd-lite':
    net = create_squeezenet_ssd_lite(len(class_names), is_test=True)
else:
    print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
net.load(model_path)

if net_type == 'vgg16-ssd':
    predictor = create_vgg_ssd_predictor(net, candidate_size=200)
elif net_type == 'mb1-ssd':
    predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)
elif net_type == 'mb1-ssd-lite':
    predictor = create_mobilenetv1_ssd_lite_predictor(net, candidate_size=200)
elif net_type == 'mb2-ssd-lite' or net_type == "mb3-large-ssd-lite" or net_type == "mb3-small-ssd-lite":
    predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200)
elif net_type == 'sq-ssd-lite':
    predictor = create_squeezenet_ssd_lite_predictor(net, candidate_size=200)
else:
    predictor = create_vgg_ssd_predictor(net, candidate_size=200)

orig_image = cv2.imread(image_path)
image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
boxes, labels, probs = predictor.predict(image, 10, 0.4)

for i in range(boxes.size(0)):
    box = boxes[i, :]
    cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)
    #label = f"""{voc_dataset.class_names[labels[i]]}: {probs[i]:.2f}"""
    label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
    cv2.putText(orig_image, label,
                (box[0] + 20, box[1] + 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,  # font scale
                (255, 0, 255),
                2)  # line type
plt.imshow(orig_image)

In [ ]:
#video
import time
import imutils
from PIL import Image
from IPython.display import clear_output

def draw_boxes(image, boxes, labels, probs):
    for i in range(boxes.size(0)):
        box = boxes[i, :]
        cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 255, 0), 4)
        label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
        cv2.putText(image, label,
                    (int(box[0]) + 20, int(box[1]) + 40),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,  # font scale
                    (255, 0, 255),
                    2)  # line type
    
video_file = '/content/drive/MyDrive/deep learning/videoplayback.mp4'
cap = cv2.VideoCapture(video_file)

if (cap.isOpened() == False):
    print('Error while trying to read video. Please check path again')

frame_count = 0 # to count total frames
total_fps = 0 # to get the final frames per second
#while(cap.isOpened()):                      # read until end of video
while(frame_count<200):
    ret, image = cap.read()                 # capture each frame of the video
    if ret == False:
        break
    #image = imutils.resize(image, width=700)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    start_time = time.time()      
    boxes, labels, probs = predictor.predict(image, 10, 0.4)
    end_time = time.time()
    draw_boxes(image, boxes, labels, probs)
    fps = 1 / (end_time - start_time)
    total_fps += fps
    fig=plt.figure(figsize=(12, 8))
    clear_output(wait=True)
    plt.imshow(image)
    plt.show()
    print("frame {}, FPS : {:.3f}".format(frame_count, fps))
    frame_count += 1
    wait_time = max(1, int(fps/4))
cap.release()
if(frame_count==0):
    print("Erreur video")
else:
    avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")

apprentissage avec FDDB 

In [ ]:
!python3 train_ssd.py --datasets '/content/drive/MyDrive/FDDB_2010' --validation_dataset '/content/drive/MyDrive/FDDB_2010' --net mb2-ssd-lite --pretrained_ssd '/content/drive/MyDrive/deep learning/MobileNetSSD/models/mb2-ssd-lite-mp-0_686.pth' --num_epochs 70 --scheduler cosine --lr 0.01 --t_max 200 --validation_epochs 5

evalution le model

In [ ]:
!python eval_ssd.py --net mb2-ssd-lite  --dataset '/content/drive/MyDrive/FDDB_2010' --trained_model '/content/drive/MyDrive/deep learning/MobileNetSSD/models/mb2-ssd-lite-Epoch-45-Loss-1.0989911688698664.pth' --label_file '/content/drive/MyDrive/deep learning/MobileNetSSD/models/face-model-labels.txt' 

test model

In [ ]:
import torch
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.utils.misc import Timer
import cv2
import matplotlib.pyplot as plt

In [ ]:
model_path = '/content/drive/MyDrive/deep learning/MobileNetSSD/models/mb2-ssd-lite-Epoch-45-Loss-1.0989911688698664.pth'
label_path = '/content/drive/MyDrive/deep learning/MobileNetSSD/models/face-model-labels.txt'
image_path = '/content/drive/MyDrive/deep learning/MobileNetSSD/facedetec.jpeg'

class_names = [name.strip() for name in open(label_path).readlines()]

net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)

net.load(model_path)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200, device=DEVICE)

image = cv2.imread(image_path)
#cv2.imshow('Image', image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

boxes, labels, probs = predictor.predict(image, 15, 0.4)

for i in range(boxes.size(0)):
    box = boxes[i, :]
    print(box)
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)
    #label = f"""{voc_dataset.class_names[labels[i]]}: {probs[i]:.2f}"""
    label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
    cv2.putText(image, label,
                (box[0] + 20, box[1] + 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,  # font scale
                (0, 255, 255),
                1)  # line type
path = "run_ssd_example_output.jpg"
cv2.imwrite(path, image)
print(f"Found {len(probs)} objects. The output image is {path}")
plt.figure(figsize=(15,15))
plt.imshow(image)

# implementation reconnaisance et detection

In [ ]:
#import vision
import torch
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.utils.misc import Timer
import cv2
import matplotlib.pyplot as plt

import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
#from torchsummary import summary
from __future__ import print_function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os

In [ ]:
#create a CNN
#avec deux couche de convolution, trois couche complètement connectée, deux couche de pooling
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.name = "CNN for face"
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2,2)

        #self.conv3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5)
        #self.pool3 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(32*22*22,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))    # input(3, 100, 100) output(16, 96, 96)
        x = self.pool1(x)            # output(16, 48, 48)
        x = F.relu(self.conv2(x))    # output(32, 44, 44)
        x = self.pool2(x)            # output(32, 22, 22)
        #x = F.relu(self.conv3(x))    # output(48, 18, 18)
        #x = self.pool3(x)            # output(48, 9, 9)

        x = torch.flatten(x, 1)     #aplatissement

        x = F.relu(self.fc1(x))      # output(120)
        x = F.relu(self.fc2(x))      # output(84)
        x = self.fc3(x)          # output(10)
        return x

#root
train_dir = './imageLUO'

#size
im_w, im_h = 100, 100
transform = transforms.Compose([
                 transforms.Resize((im_w, im_h)),
                 transforms.ToTensor()   #car la transformation qui suit porte sur des tenseurs
                ])

#import data
full_dataset = datasets.ImageFolder(train_dir, transform=transform)
#split dataset for train/test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#batch et loader data
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

#put data into graphics card
#device = "cpu" #ou "cuda"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
summary(model, (3, im_w, im_h))
optimizer = optim.Adam(model.parameters(), lr=0.001)    #utilisé par l'apprentissage (lr <-> learning rate)
criterion = nn.CrossEntropyLoss()       #critère d'erreur en sortie (utilisé par l'apprentissage et le test)

#function traning
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0.0
    correct = 0
    data_len = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()                              #initialisation des gradients
        output = model(data)                               #activation du réseau
        loss = criterion(output, target)                   #calcul de l'erreur de sortie
        loss.backward()                                    #rétro-propagation du gradient
        optimizer.step()                                   #adaptation des poids des neurones
        train_loss += loss.item()# * data.size(0)

        pred = output.argmax(dim=1, keepdim=True)          # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{}]\tTrain loss: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, loss.item()))
    train_loss /= data_len
    accuracy = 100. * correct / data_len
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain accuracy: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, 100. * batch_idx / data_len, accuracy))
    return train_loss, accuracy

#function test
def test(model, device, test_loader):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target)#.item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average test loss: {:.4f}, Test accuracy: ({:.2f}%)\n'.format(loss, accuracy))
    return loss, accuracy

In [ ]:
model_path ='./models/mb2-ssd-lite-Epoch-99-Loss-0.8589383926656511.pth'
label_path = './models/face-model-labels.txt'

cap = cv2.VideoCapture(0)
cap.read()
class_names = [name.strip() for name in open(label_path).readlines()]

net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)

net.load(model_path)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200, device=DEVICE)

timer = Timer()

model = Net()                                    # re-création d'un réseau de neurones (identique au précédent)
model.load_state_dict(torch.load('./models/save_model/best_model.pth'))    # chargement du modèle
model.eval()

def predict(model, image):
        #image = cv2.resize(image, (100, 100))
        np_img = np.array(image)                        # conversion en tableau numpy
        image_tensor = torch.from_numpy(np_img)         # conversion en tenseur PyTorch
        image_tensor = image_tensor.permute(2, 0, 1)    # nouvel ordre des dimensions
        image_tensor = image_tensor.reshape([1, 3, np_img.shape[0], np_img.shape[1]]) # ajout d'une dimension (cf + haut)
        output = model(image_tensor.float())        # application de l'image en entrée du réseau
        index = output.data.cpu().numpy().argmax()
        return index

while True:
    ret, image = cap.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    timer.start()
    boxes, labels, probs = predictor.predict(image, 15, 0.4)
    interval = timer.end()
    print('Time: {:.2f}s, Detect Objects: {:d}.'.format(interval, labels.size(0)))

    for i in range(boxes.size(0)):
          box = boxes[i, :]
          print(box)
          cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3]), (255, 255, 0), 4)
          if box[1]>0 and box[0]>0 and box[2]>0 and box[3]>0:
              rec = image[int(box[1]):int(box[3]),int(box[0]):int(box[2])]
              #programme principal
              #classe "Emmanuel" : incdex = 0 classe "HUANGXinjian" : index = 1 ; classe "LUOWenqi" ; index = 2 ：classe "Valex" index = 3 : classe "unknown" index = 4

              index = predict(model, rec)
              print('index : ', index)
              #affichage du résultat sur l'image
              classe = []    #nom des classes
              classe.append('Emmanuel')
              classe.append('HUANGXingjian')
              classe.append('LUOWenqi')
              classe.append('Valex')
              classe.append('unknown')
              test_loss, test_accuracy = test(model, device, test_loader)
              label = f"{str(classe[index])}: {test_accuracy:.2f}"
              cv2.putText(image, label,
                            (int(box[0]) + 20, int(box[1]) + 40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,  # font scale
                            (0, 0, 255),
                            1)  # line type
    cv2.imshow('annotated', image)
    print(f"Found {len(probs)} objects.")
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# test communication SPI

In [ ]:
#pour Raspberry PI4
import spidev

spi=spidev.SpiDev()
spi.open(0,0)
spi_max_speed_hz=976000

while True:
    to_send = [0]
    print(to_send)
    spi.writebytes(to_send)

In [ ]:
# pour MSP430
#include <msp430.h> 
unsigned char carac;
void Init_SPI()
{
    UCB0CTL1 = UCSWRST;
    P1DIR &=  ~BIT4; // CS input
    P1OUT |=  BIT4; // CS high
    P1SEL |= BIT7 | BIT5;  // Enable SIMO + SCLK
    P1SEL2 |= BIT7 | BIT5; // Enable SIMO + SCLK
    UCB0CTL0 |= UCCKPH | UCMSB | UCSYNC; // 3-pin, 8-bit SPI master
    UCB0CTL0 &= ~UCMST;
    UCB0CTL1 |= UCSSEL_2; // SMCLK
    UCB0CTL1 &= ~UCSWRST;
    UCB0BR0 = 26;
    UCB0BR1 = 0;
    IE2|= UCB0RXIE;
}
void RXSPI(unsigned char *c)
{
    while (!(IFG2 & UCB0RXIFG));              // buffer Rx USCI_B0 plein ?
    *c = UCB0RXBUF;
}
void main(void ){
    WDTCTL = WDTPW + WDTHOLD;
    BCSCTL1= CALBC1_1MHZ; //frequence d’horloge 1MHz
    DCOCTL= CALDCO_1MHZ;
    Init_SPI();
    P1SEL |= BIT0; // selection fonction TA0.1
    P1SEL2 &= ~BIT0; // selection fonction TA0.1
    P1DIR |= BIT6; // P1.6 en sortie
    P1OUT &= ~BIT6;
    //unsigned char c;
    __enable_interrupt();
    while(1);
}
#pragma vector=USCIAB0RX_VECTOR
__interrupt void USCI0RX_ISR(void)
{
    while (!(IFG2 & UCB0RXIFG));
    carac = UCB0RXBUF;
    if(carac==0xED)
    {
       P1OUT |= BIT6;
       //__delay_cycles(100000);
    }
    else
    {
       P1OUT &= ~BIT6;
    }
    IFG2 &= ~UCA0RXIFG; // Clear RX flag
}

# code de capture ultrason

In [ ]:
//------------------------------------------------------------------------------
// DUAL-US Display 2 US sensors JYC-2022
//------------------------------------------------------------------------------

#include <msp430g2553.h> // specific part of msp430.h
#include <intrinsics.h> // interrupt, delay, power mode ...
#include <stdbool.h> // boolean definitions
#include "moteur.h"

unsigned int memo_capt1, memo_capt2;
unsigned int diff_capt1, diff_capt2;
unsigned int distance;

//-- Timer Interrupt Service Routine -------------------------------------------
#pragma vector = TIMER0_A0_VECTOR
__interrupt void Timer_ISR(void)
{
unsigned int capt;

 capt = TA0CCR0;
TA0CCTL0 ^= CM_3;

 diff_capt1 = capt - memo_capt1;
memo_capt1 = capt;
}

//-- Main ----------------------------------------------------------------------
void main(void)
{
WDTCTL = WDTPW | WDTHOLD; // stop watch-dog timer

 //Aff_Init();
init();
P1DIR = BIT0;
P1OUT &= ~BIT0;

 // set SMCLK (Sub Main Clock) to 1MHz
DCOCTL = CALDCO_1MHZ; BCSCTL1 = CALBC1_1MHZ;

 //.. Timer0 as signal measure ................................................
P1SEL |= BIT1; // set P1.1 as TimerA-0 capture CCI0A
P1SEL2 &= ~BIT1; // ...
P1DIR &= ~BIT1; // set P1.1 as TimerA-0 input

 // set TimerA-0 to count at 1MHz
TA0CTL = TASSEL_2 | ID_0 | MC_2 | 0;

 // set TimerA-0 as rise edge / CCI0A / Capture mode / Interrupt enable
TA0CCTL0 = CM_1 | CCIS_0 | CAP | CCIE;

 P2SEL &= ~BIT7; // set P2.7 as I/O
P2SEL2 &= ~BIT7;
P2DIR |= BIT7; // set P2.7 as output
P2OUT &= ~BIT7; // set P2.7 to false

memo_capt1 = memo_capt2 = 0;
diff_capt1 = diff_capt2 = 0;

 //............................................................................
__enable_interrupt();

 while(true) // endless loop
{
unsigned int
distance = diff_capt1 / 58;
unsigned int i = 0;
//Aff_valeur(convert_Hex_Dec(diff_capt1 >> 8));

 P2OUT |= BIT7; // set P1.0 to false
__delay_cycles(1200); // delay 1ms
P2OUT &= ~BIT7; // set P1.0 to false
__delay_cycles(300000); // delay 250ms
if(diff_capt1 < 836){
for(i=0 ;i<60000;i++){
    reculer();
}
for(i=0 ;i<50000;i++){
                       tourner_droite();}
avancer();
P1OUT |= BIT0;
}
else if(diff_capt1 < 1672){
    for(i=0 ;i<50000;i++){
                           tourner_droite();}
    avancer();
    P1OUT |= BIT0;
}
else {
    avancer();
    P1OUT &= ~BIT0;
}
 }
}

# code de MSP430

In [ ]:
#include <msp430.h>
#include <math.h>
#include <ADC.h>
unsigned char carac;
unsigned int memo_capt1, memo_capt2;
unsigned int diff_capt1, diff_capt2;
unsigned int distance;
void initRobot(){//Initialisation du robot (moteur, sens des roues, etc...)
    P2DIR |= (BIT5|BIT4|BIT2|BIT1);
    P2DIR &= ~(BIT3|BIT0);
    P2OUT &= ~(BIT4|BIT2);
    ADC_init();
}
void TIMER(int taccr0){
    BCSCTL1= CALBC1_1MHZ; //frequence d’horloge 1MHz
    DCOCTL= CALDCO_1MHZ; // "
    P1DIR |= BIT0; //BIT 0 du port 1 (LED de la Launchpad) en sortie
    TA0CTL = 0|(TASSEL_2 | ID_3); //source SMCLK, pas de predivision ID_0
    TA0CTL |= MC_3; //comptage en mode up
    TA0CCR0 = taccr0;
    while(!(TA0CTL & TAIFG)); //attente flag TAIFG a 1
            TA0CTL &= ~TAIFG; //RAZ TAIFG
}
void initialiseUltrason()
{
    P1SEL |= BIT1; // set P1.1 as TimerA-0 capture CCI0A
    P1SEL2 &= ~BIT1; // ...
    P1DIR &= ~BIT1; // set P1.1 as TimerA-0 input

     // set TimerA-0 to count at 1MHz
    TA0CTL = TASSEL_2 | ID_0 | MC_2 | 0;

     // set TimerA-0 as rise edge / CCI0A / Capture mode / Interrupt enable
    TA0CCTL0 = CM_1 | CCIS_0 | CAP | CCIE;

    P2SEL &= ~BIT7; // set P2.7 as I/O
    P2SEL2 &= ~BIT7;
    P2DIR |= BIT7; // set P2.7 as output
    P2OUT &= ~BIT7; // set P2.7 to false
}
void attenteInstruction(float dureeAttente){
    int entierSeconde = floor(dureeAttente);
    float sec_dec = dureeAttente-entierSeconde;
    int i=0;
    for(i=0; i<entierSeconde; i++){
        TIMER(62500);
    }
    int taccr0 = 62500*sec_dec;
    TIMER(taccr0);
}
void PWM(int gauche, int droite){//Gestion de la PWM
    P2DIR |= (BIT4|BIT2);
    P2OUT&=~ (BIT4|BIT2);
    TA1CTL = (TASSEL_2 | ID_0 | MC_1) ;
    TA1CCR0 = 200; // determine la periode du signal
    TA1CCR1 = gauche; // determine le rapport cyclique du signal
    TA1CCR2 = droite;
    TA1CCTL1 = OUTMOD_7;
    TA1CCTL2 = OUTMOD_7;
    P2SEL |= (BIT4|BIT2);
    P2SEL2 &= ~ (BIT4|BIT2);
}
void initCapteurDroit(){ //init capteur de droite
    P1DIR&=~BIT4;
    P1REN|=BIT4;
    P1OUT&=~BIT4;
}
void initCapteurGauche(){ //init capteur de gauche
    P1DIR&=~BIT7;
    P1REN|=BIT7;
    P1OUT&=~BIT7;
}
void avancer(){
    P2OUT &=~ BIT1;
    P2OUT |= BIT5;
}
void reculer(){
    P2OUT &=~ BIT5;
    P2OUT |= BIT1;
}
void tournerGauche(){
    P2OUT |= BIT5;
    P2OUT |= BIT1;
}
void tournerDroite(){
    P2OUT &=~ BIT5;
    P2OUT &=~ BIT1;
}
void arret(){
    PWM(0,0);
}
void demiTour(){
    tournerGauche();
    attenteInstruction(1);
}
void Init_SPI()
{
    UCB0CTL1 = UCSWRST;
    P1DIR &=  ~BIT4; // CS input
    P1OUT |=  BIT4; // CS high
    P1SEL |= BIT7 | BIT5;  // Enable SIMO + SCLK
    P1SEL2 |= BIT7 | BIT5; // Enable SIMO + SCLK
    UCB0CTL0 |= UCCKPH | UCMSB | UCSYNC; // 3-pin, 8-bit SPI master
    UCB0CTL0 &= ~UCMST;
    UCB0CTL1 |= UCSSEL_2; // SMCLK
    UCB0CTL1 &= ~UCSWRST;
    UCB0BR0 = 26;
    UCB0BR1 = 0;
}
void RXSPI(unsigned char *c)
{
    while (!(IFG2 & UCB0RXIFG));              // buffer Rx USCI_B0 plein ?
    *c = UCB0RXBUF;
}
void main(void ){
    WDTCTL = WDTPW + WDTHOLD;
    BCSCTL1= CALBC1_1MHZ; //frequence d’horloge 1MHz
    DCOCTL= CALDCO_1MHZ;
    P1SEL |= BIT0; // selection fonction TA0.1
    P1SEL2 &= ~BIT0; // selection fonction TA0.1
    P1DIR |= BIT6; // P1.6 en sortie
    P1OUT &= ~BIT6;
    //initialisation fonctions robot et capteurs
    initRobot();
    initCapteurDroit();
    initCapteurGauche();
    Init_SPI();
    memo_capt1 = memo_capt2 = 0;
    diff_capt1 = diff_capt2 = 0;
    __enable_interrupt();
    //CODE ROBOT
    //PWM(80,80);
    //avancer();
    //attenteInstruction(0.7);
    //arret();
    //attenteInstruction(1);
    unsigned char c;
    while(1){
        PWM(80,80);
        avancer();
        unsigned int distance = diff_capt1 / 58;
        unsigned int i = 0;
        //Aff_valeur(convert_Hex_Dec(diff_capt1 >> 8));

         P2OUT |= BIT7; // set P1.0 to false
        __delay_cycles(1200); // delay 1ms
        P2OUT &= ~BIT7; // set P1.0 to false
        __delay_cycles(300000); // delay 250ms
        if(diff_capt1 < 836){
            for(i=0 ;i<15000000;i++){
                arret();
                while (!(IFG2 & UCB0RXIFG));
                carac = UCB0RXBUF;
                if(carac==0x00||carac==0x01||carac==0x02||carac==0x03||carac==0x04)
                {
                   P1OUT |= BIT6;
                   //__delay_cycles(100000);
                }
                else
                {
                   P1OUT &= ~BIT6;
                }
            }
            PWM(80,80);
            tournerGauche();
            __delay_cycles(100000);
            PWM(80,80);
            avancer();
        }
        else{
            PWM(80,80);
            avancer();
        }
    }
}
#pragma vector = TIMER0_A0_VECTOR
__interrupt void Timer_ISR(void)
{
unsigned int capt;

 capt = TA0CCR0;
TA0CCTL0 ^= CM_3;

 diff_capt1 = capt - memo_capt1;
memo_capt1 = capt;
}

# code de RaspberryPI4 final

In [ ]:
#import vision
import torch
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.utils.misc import Timer
import cv2
import matplotlib.pyplot as plt

import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
#from torchsummary import summary
from __future__ import print_function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
import spidev
import time

#create a CNN
#avec deux couche de convolution, trois couche complètement connectée, deux couche de pooling
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.name = "CNN for face"
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2,2)

        #self.conv3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5)
        #self.pool3 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(32*22*22,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))    # input(3, 100, 100) output(16, 96, 96)
        x = self.pool1(x)            # output(16, 48, 48)
        x = F.relu(self.conv2(x))    # output(32, 44, 44)
        x = self.pool2(x)            # output(32, 22, 22)
        #x = F.relu(self.conv3(x))    # output(48, 18, 18)
        #x = self.pool3(x)            # output(48, 9, 9)

        x = torch.flatten(x, 1)     #aplatissement

        x = F.relu(self.fc1(x))      # output(120)
        x = F.relu(self.fc2(x))      # output(84)
        x = self.fc3(x)          # output(10)
        return x

#root
train_dir = './imageLUO'

#size
im_w, im_h = 100, 100
transform = transforms.Compose([
                 transforms.Resize((im_w, im_h)),
                 transforms.ToTensor()   #car la transformation qui suit porte sur des tenseurs
                ])

#import data
full_dataset = datasets.ImageFolder(train_dir, transform=transform)
#split dataset for train/test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#batch et loader data
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

#put data into graphics card
#device = "cpu" #ou "cuda"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
summary(model, (3, im_w, im_h))
optimizer = optim.Adam(model.parameters(), lr=0.001)    #utilisé par l'apprentissage (lr <-> learning rate)
criterion = nn.CrossEntropyLoss()       #critère d'erreur en sortie (utilisé par l'apprentissage et le test)

#function traning
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0.0
    correct = 0
    data_len = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()                              #initialisation des gradients
        output = model(data)                               #activation du réseau
        loss = criterion(output, target)                   #calcul de l'erreur de sortie
        loss.backward()                                    #rétro-propagation du gradient
        optimizer.step()                                   #adaptation des poids des neurones
        train_loss += loss.item()# * data.size(0)

        pred = output.argmax(dim=1, keepdim=True)          # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{}]\tTrain loss: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, loss.item()))
    train_loss /= data_len
    accuracy = 100. * correct / data_len
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain accuracy: {:.4f}'.format(
                epoch, batch_idx * len(data), data_len, 100. * batch_idx / data_len, accuracy))
    return train_loss, accuracy

#function test
def test(model, device, test_loader):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target)#.item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average test loss: {:.4f}, Test accuracy: ({:.2f}%)\n'.format(loss, accuracy))
    return loss, accuracy

model_path ='./models/mb2-ssd-lite-Epoch-99-Loss-0.8589383926656511.pth'
label_path = './models/face-model-labels.txt'

cap = cv2.VideoCapture(0)
cap.read()
class_names = [name.strip() for name in open(label_path).readlines()]

net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)

net.load(model_path)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200, device=DEVICE)

timer = Timer()

model = Net()                                    # re-création d'un réseau de neurones (identique au précédent)
model.load_state_dict(torch.load('./models/save_model/best_model.pth'))    # chargement du modèle
model.eval()

def predict(model, image):
        #image = cv2.resize(image, (100, 100))
        np_img = np.array(image)                        # conversion en tableau numpy
        image_tensor = torch.from_numpy(np_img)         # conversion en tenseur PyTorch
        image_tensor = image_tensor.permute(2, 0, 1)    # nouvel ordre des dimensions
        image_tensor = image_tensor.reshape([1, 3, np_img.shape[0], np_img.shape[1]]) # ajout d'une dimension (cf + haut)
        output = model(image_tensor.float())        # application de l'image en entrée du réseau
        index = output.data.cpu().numpy().argmax()
        return index

spi=spidev.SpiDev()
spi.open(0,0)
spi_max_speed_hz=7629
while True:
    ret, image = cap.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    timer.start()
    boxes, labels, probs = predictor.predict(image, 15, 0.4)
    interval = timer.end()
    print('Time: {:.2f}s, Detect Objects: {:d}.'.format(interval, labels.size(0)))

    for i in range(boxes.size(0)):
          box = boxes[i, :]
          print(box)
          cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3]), (255, 255, 0), 4)
          if box[1]>0 and box[0]>0 and box[2]>0 and box[3]>0:
              rec = image[int(box[1]):int(box[3]),int(box[0]):int(box[2])]
              #programme principal
              #classe "Emmanuel" : incdex = 0 classe "HUANGXinjian" : index = 1 ; classe "LUOWenqi" ; index = 2 ：classe "Valex" index = 3 : classe "unknown" index = 4

              index = predict(model, rec)
              print('index : ', index)
              for i in range(15000000):
                        to_send = [index]
                        print(to_send)
                        spi.writebytes(to_send)

              #affichage du résultat sur l'image
              classe = []    #nom des classes
              classe.append('Emmanuel')
              classe.append('HUANGXingjian')
              classe.append('LUOWenqi')
              classe.append('Valex')
              classe.append('unknown')
              test_loss, test_accuracy = test(model, device, test_loader)
              label = f"{str(classe[index])}: {test_accuracy:.2f}"
              cv2.putText(image, label,
                            (int(box[0]) + 20, int(box[1]) + 40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,  # font scale
                            (0, 0, 255),
                            1)  # line type
    cv2.imshow('annotated', image)
    print(f"Found {len(probs)} objects.")
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()